In [1]:
using LinearAlgebra
using Pipe: @pipe

In [2]:
vertices(E) = vcat(E[:, 1], E[:, 2]) |> unique!
function alledges(V) # this would be a one-liner if julia's 
    n = length(V) # array comprehension were better
    V̄ = Array{Array{Float64, 1}, 2}(undef, 0,2)
    for i in 1:n-1, j in i+1:n
        V̄ = vcat(V̄, [[V[i]] [V[j]]])
    end
    return V̄
end
function sortedges(E; weight=norm, rev=false)
    E = [[E[i, 1], E[i, 2]] for i in 1:size(E, 1)] # convert to array
    E = sort!(E, by = e -> weight(e[1] - e[2]), rev=rev) # sort array
    E = @pipe E |> hcat.(_) |> permutedims.(_) |> vcat(_...) # convert back to matrix
    return E
end

sortedges (generic function with 1 method)

In [3]:
function spantree(E; weight=norm, rev=false) # kruskal algorithm
    E = sortedges(E; weight=weight, rev=rev)
    V = vertices(E); n = size(V, 1)
    W = W_new = Array{Array{Float64, 1}, 1}(undef, 0)
    F = F_new = Array{Array{Float64, 1}, 2}(undef, 0,2)
    k = 1
    while size(F, 1) < n - 1
        F_new = vcat(F, permutedims(E[k, :]))
        W_new = vertices(F_new)
        if size(F_new, 1) == size(W_new, 1) - 1
            F = F_new
            W = W_new
            k = 1
        else 
            k += 1
        end
    end
    return F
end

spantree (generic function with 1 method)

In [4]:
function cluster(V, k) # currently just does all edges
    T = V |> alledges |> spantree |> sortedges
    return T[1:size(V, 1) - k, :] # choose the |V|-k shortest edges
end

cluster (generic function with 1 method)

In [5]:
using Plots, PlotThemes
theme(:dark)
plotlyjs()

┌ Info: Precompiling PlotlyJS [f0f68f2c-4968-5e81-91da-67840de0976a]
└ @ Base loading.jl:1278


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-12301424950870151994\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16710836592400067579\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-12301424950870151994\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Plots.PlotlyJSBackend()

In [6]:
m = 3 # dimension
V = [rand(m) for i in 1:10] # random points
E = cluster(V, rand(2:size(V, 1))) # choose a randomly sized cluster
V̄ = [getindex.(V, j) for j in 1:m] # separate x, y, z components for plotting

3-element Array{Array{Float64,1},1}:
 [0.16394280345465329, 0.1544842672623803, 0.08608916949680978, 0.4204540789605291, 0.5831999738235891, 0.2998338572366228, 0.7726858723313808, 0.4630087988800766, 0.046243542634913704, 0.24220081059731857]
 [0.8791592842020837, 0.0800492484391837, 0.6291168670120788, 0.08202728726692299, 0.487686958149385, 0.6344506158312067, 0.4994632886993695, 0.5097374730831594, 0.7026631495665954, 0.7597177734783795]
 [0.5636208904597477, 0.8772693627569552, 0.5895226835618761, 0.7834035623660809, 0.2520386195053832, 0.4138970052449622, 0.7304596889075163, 0.20745185943557876, 0.6427486237158215, 0.23796145732675256]

In [7]:
fig = scatter(V̄..., leg=false)
for i in 1:size(E, 1)
    Ē = [getindex.(E[i, :], j) for j in 1:m]
    plot!(fig, Ē...)
end
display(fig)

<!DOCTYPE html>
 
 
 Plots.jl